# Fine-tuning with Low-Rank Adaptation (LoRA)

In [ ]:
!pip install wandb python-dotenv datasets peft

In [7]:
import wandb
from dotenv import load_dotenv

from peft import LoraConfig, TaskType, get_peft_model
import transformers
from transformers import BertForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

transformers.logging.set_verbosity_error()

In [8]:
load_dotenv()
wandb.login()

True

## Dataset loading

In [9]:
train_data = (load_dataset('csv', data_files='../data/train.csv', split='train')
              .class_encode_column('is_humor')
              .rename_column('is_humor', 'labels'))

dev_data = (load_dataset('csv', data_files='../data/dev.csv', split='train')
            .class_encode_column('is_humor')
            .rename_column('is_humor', 'labels'))

test_data = (load_dataset('csv', data_files='../data/test.csv', split='train')
             .class_encode_column('is_humor')
             .rename_column('is_humor', 'labels'))

## Model setup

In [10]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

In [11]:
train_data = train_data.map(tokenize, batched=True)
dev_data = dev_data.map(tokenize, batched=True)
test_data = test_data.map(tokenize, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [12]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

## LoRA Setup 

In [13]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=1,
    lora_alpha=1,
    lora_dropout=0.1,
    bias="none"
)

In [14]:
peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

trainable params: 38,402 || all params: 109,522,180 || trainable%: 0.035063217331868304


In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    # accuracy = accuracy_score(labels, predictions)
    # precision = precision_score(labels, predictions, average='weighted')
    # recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    
    return {
        # 'accuracy': accuracy,
        # 'precision': precision,
        # 'recall': recall,
        'f1': f1
    }
    

## Training setup

In [16]:
training_arguments = TrainingArguments(
    output_dir='bert-lora-humor-detection',
    evaluation_strategy="epoch",
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    report_to="wandb",
    run_name="bert-lora-humor-01"
)

In [17]:
trainer = Trainer(
    model=peft_model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=dev_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
%%time
trainer.train()

In [26]:
trainer.predict(test_data).metrics

{'test_loss': 0.3577665686607361,
 'test_f1': 0.8537736197348548,
 'test_runtime': 37.9098,
 'test_samples_per_second': 21.103,
 'test_steps_per_second': 2.638}